### Import Lib

In [2]:
import os
from tqdm import tqdm
from glob import glob
import xml.etree.ElementTree as ET

In [4]:
def create_yolo_ann_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [5]:
all_dirs = os.listdir(os.path.join(os.getcwd(),"Custom_dataset"))
all_dirs

['Bicycle', 'Bike', 'Bus', 'Car', 'Cng', 'Truck']

In [10]:
for item in all_dirs:
    path = os.path.join(os.getcwd(),"Custom_dataset",item,"yolo_annotations")
    print(path)
    create_yolo_ann_folder(path)

C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Bicycle\yolo_annotations
C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Bike\yolo_annotations
C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Bus\yolo_annotations
C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Car\yolo_annotations
C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Cng\yolo_annotations
C:\Users\Pc-Klinik\Desktop\Object_detection_Thesis\Custom_dataset\Truck\yolo_annotations


In [60]:
dir_dict = {}
for item in all_dirs:
    annotations = glob(os.path.join(os.getcwd(),"Custom_dataset",item,"annotations","*.xml"))
    images = glob(os.path.join(os.getcwd(),"Custom_dataset",item,"images","*.[jp][pn]g"))
    
    assert len(annotations) == len(images), f"""
                                            Opps! Some files are missing \n 
                                            ITEM:{item}, Annotation lenth:{len(annotations)} & Image length:{len(images)}
                                            """
    dir_dict[item] = {}
    dir_dict[item]["annotations"] = annotations

dir_dict

{'Bicycle': {'annotations': ['C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (1).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (10).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (100).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (101).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (102).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (103).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (104).xml',
   'C:\\Users\\Pc-Klinik\\Desktop\\Object_detection_Thesis\\Custom_dataset\\Bicycle\\annotations\\bicycle (105).xml',
   'C:\\Users\\Pc-Klinik\\Desktop

### Finding extra annotation files

In [6]:
a = sorted(os.listdir(os.path.join(os.getcwd(),'Custom_dataset','Truck','yolo_annotations')))
b = sorted(os.listdir(os.path.join(os.getcwd(),'Custom_dataset','Truck','images')))

if 'desktop.ini' in a:
    a.remove('desktop.ini')
if 'desktop.ini' in b:
    b.remove('desktop.ini')
    
a1= []; b1=[]

for i in range(len(a)): 
    n = a[i]
    num1 = int(n[n.find('(')+1 : n.find(')')])
    a1.append(num1)
        

for i in range(len(b)):
    m = b[i]
    num2 = int(m[m.find('(')+1 : m.find(')')])
    b1.append(num2)
    
for i in a1:
    if i not in b1:
        print(i)

722
736


In [62]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(cls_id, path, annotations):

    for in_path in tqdm(annotations):
        basename = os.path.basename(in_path)
        basename_no_ext = os.path.splitext(basename)[0]
        
        try:
            out_path = os.path.join(path, basename_no_ext + '.txt')
            out_file = open(out_path, 'w')
        
            tree = ET.parse(in_path)
            root = tree.getroot()
            size = root.find('size')
            w = int(size.find('width').text)
            h = int(size.find('height').text)

            for obj in root.iter('object'):
                xmlbox = obj.find('bndbox')
                b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
                bb = convert((w,h), b)
                out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
            
            out_file.close()
            
        except:
            print(f"filename:{basename}")

In [63]:
for idx, key in enumerate(all_dirs):
    path = os.path.join(os.getcwd(),"Custom_dataset",key,"yolo_annotations")
    annotations = dir_dict[key]['annotations']
    convert_annotation(idx, path, annotations)
    
    

100%|██████████████████████████████████████████████████████████████████████████████| 728/728 [00:00<00:00, 1203.93it/s]
